In [1]:
import pandas as pd
import os

In [2]:
file_name = os.path.join("Users","liorsi","Downloads","db.pkl")

In [6]:
df = pd.read_pickle("db.pkl")

In [122]:
test_df = pd.read_pickle("oded.pkl")

In [123]:
test_df

,Last key-,Current key-,HD-,RPD-,PPD-
0,-1,60,176.0,0.0,177.0
1,60,17,72.0,-40.0,31.0
2,17,4,137.0,-24.0,113.0
3,4,34,76.0,7.0,83.0
4,34,1,117.0,17.0,134.0
...,...,...,...,...,...
95,49,51,67.0,256.0,323.0
96,51,43,88.0,122.0,210.0
97,43,49,64.0,757.0,821.0
98,49,5,48.0,297.0,345.0


In [ ]:
#TODO:
## -1 to null

In [105]:

# HD - hold duration
# RPD - time between release to current
# PPD - time between two press = HD + RPD #TO Remove


In [124]:


def min_max_norm(df_, col_name, min_val = -700, max_val = 700):
    return (df_[col_name] - min_val) / (max_val - min_val)

for df_to_norm in [df, test_df]:
    for col in ["HD-","RPD-"]:
        df_to_norm[f"{col}_norm"] = min_max_norm(df_to_norm, col)
        df_to_norm[f"{col}_norm"] = df_to_norm[f"{col}_norm"].apply(lambda x: 1 if x > 1 else x )
        df_to_norm[f"{col}_norm"] = df_to_norm[f"{col}_norm"].apply(lambda x: 0 if x < 0 else x )

In [125]:
from sklearn.preprocessing import OneHotEncoder

In [126]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df[['Last key-',"Current key-"]])

OneHotEncoder(handle_unknown='ignore')

In [127]:
test_df

,Last key-,Current key-,HD-,RPD-,PPD-,HD-_norm,RPD-_norm
0,-1,60,176.0,0.0,177.0,0.625714,0.500000
1,60,17,72.0,-40.0,31.0,0.551429,0.471429
2,17,4,137.0,-24.0,113.0,0.597857,0.482857
3,4,34,76.0,7.0,83.0,0.554286,0.505000
4,34,1,117.0,17.0,134.0,0.583571,0.512143
...,...,...,...,...,...,...,...
95,49,51,67.0,256.0,323.0,0.547857,0.682857
96,51,43,88.0,122.0,210.0,0.562857,0.587143
97,43,49,64.0,757.0,821.0,0.545714,1.000000
98,49,5,48.0,297.0,345.0,0.534286,0.712143


In [128]:
def encode_features(df_,ohe_enc):
    keys_features_np = ohe_enc.transform(df_[['Last key-',"Current key-"]]).toarray()
    keys_features_df = pd.DataFrame(keys_features_np, columns = ohe_enc.get_feature_names_out())
    return pd.concat([df_[["HD-_norm","RPD-_norm"]].reset_index(drop = True),keys_features_df],axis = 1)

In [129]:
features = encode_features(df,enc )
test_features = encode_features(test_df,enc )

In [84]:
from sklearn.svm import OneClassSVM
clf = OneClassSVM(gamma='auto').fit(features[:2000].values)

In [118]:
ocsvm_scores_test = clf.score_samples(features[2000:].values)
ocsvm_scores_train = clf.score_samples(features[:2000].values)

In [120]:
ocsvm_scores_test.mean() 

949.6686146635217

In [130]:
clf.score_samples(test_features.values).mean()

956.1041922963523

In [102]:
from sklearn.ensemble import IsolationForest
isof = IsolationForest(random_state = 0).fit(features[:2000].values)

In [113]:
isof_scores_test = isof.score_samples(features[2000:].values)
isof_scores_train = isof.score_samples(features[:2000].values)

In [114]:
isof_preds_train.mean()

-0.3554913742621293

In [131]:
 isof.score_samples(test_features.values).mean()

-0.3378881949337486